# Image data - 5-way classifier
# Shap implementation.

In [ ]:
!pip install shap

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm as tqdm
import random
import seaborn as sns
import os
import glob
import tensorflow as tf
from sklearn.metrics import classification_report, roc_curve, auc
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.models import Model, load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from PIL import Image
import shap
%matplotlib inline
from PIL import Image

In [ ]:
img_dpi= 300

# set results paths

In [ ]:
path_res = Path("/project/restored-files/DiseaseClassifierPaper2021/chemPredPD/demo_output/image_data_5way_SHAP")

In [ ]:
#code to set the path to save the model
path_models = path_res.joinpath("saved_models")
print(path_models)
path_models.is_dir()

In [ ]:
#code to set the path to save plots
path_plots = path_res.joinpath("Plots")
print(path_plots)
path_plots.is_dir()

In [ ]:
#code to create folders if not present
if not path_plots.is_dir():
    path_plots.mkdir(parents=True, exist_ok=True)
path_plots.is_dir()
if not path_models.exists():
    path_models.mkdir(parents=True, exist_ok=True)
path_models.is_dir()

# load images

In [ ]:
directory = Path("/project/restored-files/DiseaseClassifierPaper2021/chemPredPD/demo_data/Image_15000/")
#directory = Path('/tmp/Image_data/')
subdirectory = list(directory.glob('*'))
print(subdirectory)

In [ ]:
SNCAx3_paths = glob.glob(str(directory)+"/**/*3xSNCA*", recursive=True)
oligomer_paths =glob.glob(str(directory)+"/**/*oligomer*", recursive=True)
rotenone_paths =glob.glob(str(directory)+"/**/*rotenone*", recursive=True)
oligomycin_paths =glob.glob(str(directory)+"/**/*oligomycin*", recursive=True)
control_paths = glob.glob(str(directory)+"/**/*control*", recursive=True)
untreated_paths = glob.glob(str(directory)+"/**/*untreated*", recursive=True)

In [ ]:
def sample_function(): 
    return 0.5

In [ ]:
all_paths = SNCAx3_paths + oligomer_paths + rotenone_paths + oligomycin_paths + control_paths + untreated_paths
random.shuffle(all_paths, sample_function)

In [ ]:
print(len(SNCAx3_paths))
print(len(oligomer_paths))
print(len(rotenone_paths))
print(len(oligomycin_paths))
print(len(control_paths))
print(len(untreated_paths))

In [ ]:
len(all_paths)

In [ ]:
all_paths[300]

In [ ]:
Image.open(all_paths[401])

In [ ]:
labels = []
images = np.zeros((len(all_paths), 84, 84, 3))
for i, p in enumerate(tqdm(all_paths)):
    if '3xSNCA' in str(p):
        labels.append(0)
    elif 'oligomer' in str(p):
        labels.append(1)
    elif 'rotenone' in str(p):
        labels.append(2)
    elif 'oligomycin' in str(p):
        labels.append(3)
    elif 'untreated' in str(p):
        labels.append(4)
    elif 'control' in str(p):
        labels.append(4)
    image = Image.open(p)
    image = image.resize((84, 84))
    image =  np.array(image)
    images[i, :, :, :] = image
       
images = images.astype(int)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,8))
ax[0].imshow(images[1])

ax[1].imshow(images[3])

plt.show()
plt.tight_layout()

In [ ]:
print(np.unique(labels))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size = 0.2,random_state = 42             
)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size = 0.2,random_state = 42             
)

In [ ]:
print(len(X_train))
print(len(X_val))
print(len(X_test))

In [ ]:
print(len(y_train))
print(len(y_val))
print(len(y_test))

In [ ]:
nb_classes = 5
y_train_one_hot = np.eye(nb_classes)[y_train]
y_val_one_hot = np.eye(nb_classes)[y_val]
y_test_one_hot = np.eye(nb_classes)[y_test]

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(len(y_train_one_hot))
print(len(y_val_one_hot))
print(len(y_test_one_hot))

In [ ]:
batch_size = 256

In [ ]:
model = Sequential()
model.add(
Conv2D(filters=32, kernel_size=(3,3), strides=1, activation='relu', input_shape=(84, 84, 3))
)
model.add(
MaxPooling2D()
         )
model.add(
    Dropout(0.2)
)
model.add(
Conv2D(filters=32, kernel_size=(3,3), strides=1, activation='relu')
)
model.add(
MaxPooling2D()
         )
model.add(
    Dropout(0.2)
)
model.add(
Conv2D(filters=32, kernel_size=(3,3), strides=1, activation='relu')
)
model.add(
MaxPooling2D()
         )
model.add(
    Dropout(0.2)
)
model.add(
Conv2D(filters=32, kernel_size=(3,3), strides=1, activation='relu')
)
model.add(
MaxPooling2D()
         )
model.add(
    Dropout(0.2)
)
model.add(
Flatten()
)
model.add(
    Dense(10, activation="relu"))
model.add(
    Dense(5, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
)

In [ ]:
epochs = 500

In [ ]:
model_name = "Predicting disease pathway and control, 5-class"

In [ ]:
checkpoint = ModelCheckpoint(
    str(path_models/model_name)+'.hdf5', 
    monitor='val_loss', 
    verbose=1,
    save_best_only=True, 
    mode='auto', 
    save_freq='epoch'
)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=50, 
    verbose=0, 
    mode='auto',
    baseline=None, 
    restore_best_weights=True,
)

In [ ]:
history = model.fit(
    X_train,
    y_train_one_hot,
    validation_data=(X_val, y_val_one_hot),
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[early_stop,checkpoint]
)

# Plot model histories and scores

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.xlabel('Epochs')
plt.legend(frameon=False)

In [ ]:
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Test Accuracy")
plt.xlabel('Epochs')
plt.legend(frameon=False)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,7))

ax1.plot(history.history['loss'], label='Training')
ax1.plot(history.history['val_loss'], label='Validation')
ax1.legend(frameon=False)
ax1.set_title("Loss")
ax1.set_ylabel("Loss")
ax1.set_xlabel("Epochs")

ax2.plot(history.history['accuracy'], label='Training')
ax2.plot(history.history['val_accuracy'], label='Validation')
ax2.legend(frameon=False)
ax2.set_title("Accuracy")
ax2.set_ylabel("Accuracy")
ax2.set_xlabel("Epochs")

fig.suptitle(model_name);
fig.savefig(path_plots.joinpath(f"Model-perform_{os.path.basename(model_name)}.png"), bbox_inches='tight', dpi= img_dpi)

In [ ]:
model_saved = load_model(str(path_models/model_name)+'.hdf5')

In [ ]:
y_pred = model_saved.predict(X_test)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(50,20))

ax1.set_title("Count")
matrix = confusion_matrix(
    y_test_one_hot.argmax(axis=1), y_pred.argmax(axis=1)
)

matrix_norm = confusion_matrix(
    y_test_one_hot.argmax(axis=1), y_pred.argmax(axis=1),
    normalize='true',
)

ConfusionMatrixDisplay(
    confusion_matrix=matrix,
    display_labels=['3xSNCA', 'Oligomer', ' Rotenone', 'Oligomycin/Antimycin','Control']
).plot(cmap='Blues', ax=ax1, values_format="g")

ax1.set_ylabel(ax1.get_ylabel(), fontsize=40)
ax1.set_xlabel(ax1.get_xlabel(), fontsize=40, labelpad=10)
fig.axes[-1].set_visible(False)

# Saving the counts and fraction separately using extent
extent = ax1.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
# Pad the saved area by 50% in the x-direction and 20% in the y-direction

fig.savefig(path_plots.joinpath(f"CONFMATRIX_count_exp_{os.path.basename(model_name)}.png"),
            bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi, )


ax2.set_title("Fraction")
ConfusionMatrixDisplay(
    confusion_matrix=matrix_norm,
    display_labels=['3xSNCA', 'Oligomer', ' Rotenone', 'Oligomycin/Antimycin', 'Control']
).plot(cmap='Blues', ax=ax2, values_format="g")

ax2.set_ylabel(ax2.get_ylabel(), fontsize=40)
ax2.set_xlabel(ax2.get_xlabel(), fontsize=40, labelpad=10)
fig.axes[-1].set_visible(False)

extent = ax2.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
# Pad the saved area in the x-direction and y-direction

fig.savefig(path_plots.joinpath(f"CONFMATRIX_fraction_exp_{os.path.basename(model_name)}.png"),
            bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi)


fig.tight_layout()
fig.suptitle(f"CONFUSION MATRIX\n{model_name}", y=1.01)

plt.show()


print('\n\n', 
      classification_report(y_test_one_hot.argmax(axis=1), y_pred.argmax(axis=1))
     )

# SHAP

In [ ]:
background = X_train[np.random.choice(X_train.shape[0], 500, replace=False)]

In [ ]:
e = shap.DeepExplainer(model, background)

In [ ]:
X_test_new = X_test[0],X_test[11],X_test[15],X_test[2],X_test[1]

In [ ]:
X_test_new = np.asarray(X_test_new)

In [ ]:
type(X_test_new)

In [ ]:
shap_values = e.shap_values(X_test_new)

In [ ]:
X_test_new

In [ ]:
shap.image_plot(shap_values, X_test_new[0:5],  show=False)
plt.savefig(f"ShAP_{os.path.basename(model_name)}.png", dpi=img_dpi, bbox_inches='tight')

In [ ]:
most_extreme_control_indices = np.where(y_pred[:, 4] > 0.9999)[0]
most_extreme_SNCAx3_indices = np.where(y_pred[:, 0] > 0.9999)[0]
most_extreme_oligomer_indices = np.where(y_pred[:, 1] > 0.9999)[0]
most_extreme_rotenone_indices = np.where(y_pred[:, 2] > 0.9999)[0]
most_extreme_oligomycin_indices = np.where(y_pred[:, 3] > 0.9999)[0]

In [ ]:
most_extreme_control_images = X_test[most_extreme_control_indices]
most_extreme_SNCAx3_images = X_test[most_extreme_SNCAx3_indices]
most_extreme_oligomer_images = X_test[most_extreme_oligomer_indices]
most_extreme_rotenone_images = X_test[most_extreme_rotenone_indices]
most_extreme_oligomycin_images = X_test[most_extreme_oligomycin_indices]

In [ ]:
print(len(most_extreme_control_images))
print(len(most_extreme_SNCAx3_images))
print(len(most_extreme_oligomer_images))
print(len(most_extreme_rotenone_images))
print(len(most_extreme_oligomycin_images))

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(16,16))
ax[0].imshow(most_extreme_SNCAx3_images[0])
ax[1].imshow(most_extreme_oligomer_images[0])
ax[2].imshow(most_extreme_rotenone_images[0])
ax[3].imshow(most_extreme_oligomycin_images[0])
ax[4].imshow(most_extreme_control_images[0])
plt.subplots_adjust(left=0.1,
                    bottom=0.6, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[]);
plt.savefig(path_plots.joinpath(f"Most_representative_images_{os.path.basename(model_name)}.png"), dpi=img_dpi, bbox_inches='tight')
